In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import Packages.ClusteringHelper as ch
from Packages.TimeEvolving import DataEvolver
from textdistance import DamerauLevenshtein, Levenshtein
import numpy as np
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from Packages.TimeEvolving import Cluster
from tqdm.notebook import tqdm
import math
import time
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances


In [3]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
ents_data = data[data['entities'] != ''].copy()

In [4]:
ents_data = ch.add_entities_embedding(ents_data,
                                      "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\encodings")
ents_data_filtered = ents_data.copy()
documents = set(ents_data.documents)

In [9]:
evolving = DataEvolver(documents, ents_data, randomly=False, step=10)
gold_entities = []
total_clusters = []
n = 0

## Let the cycle start


In [10]:
tic = time.perf_counter()

In [ ]:
for iteration in tqdm(evolving, total=math.ceil(len(evolving.documents) / evolving.step)):
    current_mentions = list(evolving.get_current_data().mentions)
    current_encodings = list(evolving.get_current_data()['encodings'].values)
    current_entities = list(evolving.get_current_data()['entities'].values)
    current_clusters = [Cluster(mentions=[current_mentions[x]], encodings_list=[current_encodings[x]],
                                entities=[current_entities[x]]) for x in range(len(current_mentions))]

    total_clusters =  total_clusters + current_clusters


    def distance_metric(x, y):
        i, j = int(x[0]), int(y[0])  # extract indices
        sintact_distance_list = [DamerauLevenshtein().normalized_distance(el1, el2)
                                 for el1 in total_clusters[i].unique_mentions for el2 in total_clusters[j].unique_mentions]
        sintact_distance = np.mean(sintact_distance_list)
        semantic_distance = distance.cosine(total_clusters[i].encodings_mean, total_clusters[j].encodings_mean)
        return 0.63 * sintact_distance + 0.37 * semantic_distance

    X = np.arange(len(total_clusters)).reshape(-1, 1)
    m_matrix = pairwise_distances(X, X, metric=distance_metric, n_jobs=-1)
    clusterizator1 = AgglomerativeClustering(n_clusters=None, affinity='precomputed',
                                             distance_threshold=0.25,
                                             linkage="single", memory="./qua")
    cluster_numbers = clusterizator1.fit_predict(m_matrix)
    cluster_dict = {x:Cluster() for x in set(cluster_numbers)}
    for index, x in enumerate(cluster_numbers):
        cluster_dict[x] = cluster_dict[x] + total_clusters[index]
    total_clusters = total_clusters + list(cluster_dict.values())

  0%|          | 0/139 [00:00<?, ?it/s]